## Embedding model ##
>Floris Menninga \
>Datum: 8-01-2026

In [13]:
# %pip install tensorflow keras
# #%pip install tqdm
# %pip install tensorflow_text
# %pip install bs4
# %pip install joblib
# %pip install lxml

from data_parser import xml_parser
import EmbeddingModel

import io
import re
import string
import tqdm
import os

import numpy as np
import requests
import tensorflow as tf
import tensorflow_text as tf_text
import joblib
import tensorflow as tf
from tensorflow.keras import layers
import lxml

%load_ext tensorboard
AUTOTUNE = tf.data.AUTOTUNE
SEED = 0

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [14]:
os.environ['LD_LIBRARY_PATH'] = '/run/opengl-driver/lib:' + os.environ.get('LD_LIBRARY_PATH', '')
print("Gpu's: ", len(tf.config.list_physical_devices('GPU')))

Gpu's:  0


## Tokenizer

De eerste stap is het omzetten van de text in tokens:
In tegenstelling to methodes zoals bytepair encoding etc. worden de tokens gemaakt door de zinnen \
te splitten op spaties.

In [15]:
sentence = "Hallo, dit is een testzin om het programma te testen."
tokens = list(sentence.lower().split())
print(len(tokens))

10


### Vocabulary maken van deze tokens:

In [16]:
vocab, index = {}, 1 
vocab['<pad>'] = 0
for token in tokens:
  if token not in vocab:
    vocab[token] = index
    index += 1
vocab_size = len(vocab)

print(vocab)

{'<pad>': 0, 'hallo,': 1, 'dit': 2, 'is': 3, 'een': 4, 'testzin': 5, 'om': 6, 'het': 7, 'programma': 8, 'te': 9, 'testen.': 10}


### Inverse vocabulary:
Van integer index naar token:

In [17]:
inverse_vocab = {index: token for token, index in vocab.items()}

print(inverse_vocab)

{0: '<pad>', 1: 'hallo,', 2: 'dit', 3: 'is', 4: 'een', 5: 'testzin', 6: 'om', 7: 'het', 8: 'programma', 9: 'te', 10: 'testen.'}


## Vectorizeren van de zin:

In [18]:
test_sequence = [vocab[word] for word in tokens]

print(test_sequence)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


## Skip-gram maken:
Het embedding model zal gemaakt worden met behulp van het Word2Vec algorithme. Verdere uitleg over hoe dit algorithme werkt komt straks maar om dit te kunnen gebruiken 
moet er eerst een Skip-gram model gemaatk worden. 
Het belangrijkste punt van een skip-gram is dat de vector representatie gebaseerd is op de ruimtelijke nabijheid van woorden in een zin. 
Als het woord "Ketting" altijd gevolgd zou worden door het woord "zaag" zal deze combinatie van woorden vaker voorkomen dan in andere combinaties.
Een skip-gram model is een klein neural network met een inputlayer, embedding layer en output layer. 
Dit model moet een waarschijnlijkheids verdelings vector geven voor een gegeven input woord. Dat is dus de kans dat deze twee woorden samen voorkomen binnen de context lengte waarmee het model getrained is. De som van deze waarschijnlijkheids verdeling is 1.

In [19]:
window_size = 2

positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      test_sequence,
      vocabulary_size=vocab_size,
      window_size=window_size,
      negative_samples=0,
      seed=0
)

print(len(positive_skip_grams))

34


Hieronder staan vijf van de gegenereerde skip-grams. 
Zoals in de bovenstaande code gedefinieerde "window size" bestaat elk skip-gram uit twee tokens. 
In dit geval zijn het hele woorden omdat het zo getokenizeerd is. 
Later wil ik nog proberen om andere tokenizatie technieken te gebruiken zoals byte pair encoding of wordpiece / sentencepiece.


In [20]:
for target, context in positive_skip_grams[:5]:
  print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")


(3, 4): (is, een)
(1, 2): (hallo,, dit)
(8, 9): (programma, te)
(6, 8): (om, programma)
(4, 3): (een, is)


### Negative sampling

De spikgrams functie retourneerd alle positieve skip-gram paren door met een sliding window van 
een gegeven grootte over de tekst te gaan. Voor training is dit echter niet genoeg, er moeten ook negatieve samples bij zitten. Deze negatieve skip-gram paren worden verkregen door willekeurige woorden uit de vocabulary te halen en deze samen te voegen. 

Nu zal er gebruik gemaakt worden van de functie "tf.random.log_uniform_candidate_sampler"
om een aantal (num_ns) negatieve samples voor een gegeven target woord in een window te krijgen.
Voor het trainen kan het.

In [21]:
target_word, context_word = positive_skip_grams[0]

num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,
    num_true=1,
    num_sampled=num_ns,
    unique=True,
    range_max=vocab_size,
    name="negative_sampling"
)
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])


tf.Tensor([ 1  0 10  2], shape=(4,), dtype=int64)
['hallo,', '<pad>', 'testen.', 'dit']


In [22]:
squeezed_context_class = tf.squeeze(context_class, 1)

context = tf.concat([squeezed_context_class, negative_sampling_candidates], 0)

label = tf.constant([1] + [0]*num_ns, dtype="int64")
target = target_word


In [23]:
print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")


target_index    : 3
target_word     : is
context_indices : [ 4  1  0 10  2]
context_words   : ['een', 'hallo,', '<pad>', 'testen.', 'dit']
label           : [1 0 0 0 0]


In [24]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)


[0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381 0.01347162 0.01474487 0.0159558 ]


In [25]:
def generate_training_data(sequences, window_size, num_ns, vocab_size):
  targets, contexts, labels = [], [], []

  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  for sequence in tqdm.tqdm(sequences):

    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=10)

    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          name="negative_sampling")

      context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

## Word2Vec:

Het word2vec algorithme is een word embedding techniek in natural language processing die er voor zorgt dat woorden als vectors gerepresenteerd kunnen worden in een continue vector ruimte. 
Dit kan vervolgens gebruikt worden om relaties tussen woorden te achterhalen.



Hier zal de dataset ingeladen worden:

In [26]:
#text_ds = tf.data.TextLineDataset(["/run/media/floris/FLORIS_3/Data_set/PubMed/dataset_100914.txt"]).filter(lambda x: tf.cast(tf.strings.length(x), bool))
text_ds = tf.data.TextLineDataset(["/home/floris/Documenten/Dataset/trainingsData.txt"]).filter(lambda x: tf.cast(tf.strings.length(x), bool))

In [27]:
# Verwijder punctuatie / hoofdletters.
def standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Aantal woorden in seq en vocabulary grootte:
sequence_length = 200
vocab_size = 20000

# Vectorize the layer en split en map strings tokens met TextVectorization:
vectorize_layer = layers.TextVectorization(
    standardize=standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)


In [28]:
vectorize_layer.adapt(text_ds.batch(512))

In [29]:
# Inverse vocab:
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', np.str_('the'), np.str_('of'), np.str_('and'), np.str_('in'), np.str_('to'), np.str_('a'), np.str_('for'), np.str_('with'), np.str_('were'), np.str_('was'), np.str_('that'), np.str_('is'), np.str_('cancer'), np.str_('by'), np.str_('as'), np.str_('cells'), np.str_('or'), np.str_('from')]


In [30]:
# Vectoriseer de data in text_ds:
text_vector_ds = text_ds.batch(512).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [31]:
# Maak een lijst van deze gevectoriseerde data:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

6933


2026-01-29 00:03:09.125040: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [32]:
# for seq in sequences[:5]:
#   print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

In [33]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=5,
    num_ns=5,
    vocab_size=vocab_size)

# print(f"targets.shape: {targets.shape}")
# print(f"contexts.shape: {contexts.shape}")
# print(f"labels.shape: {labels.shape}")

100%|██████████| 6933/6933 [45:26<00:00,  2.54it/s]  


Gegenereerde data opslaan:
Het duurde lang om te genereren dus sla ik het op in een .joblib bestand, dit zal ik ook voor sommige vervolg stappen doen.


In [ ]:
# joblib_list = [targets, contexts, labels]

# joblib.dump(joblib_list, "targets_contexts_labels.joblib")

In [35]:
BATCH_SIZE = 256
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<_PrefetchDataset element_spec=((TensorSpec(shape=(256,), dtype=tf.int32, name=None), TensorSpec(shape=(256, 6), dtype=tf.int64, name=None)), TensorSpec(shape=(256, 6), dtype=tf.int64, name=None))>


In [36]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim)

  def call(self, pair):
    target, context = pair
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

In [37]:
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)

Hier wordt het Word2Vec algorithme gebruikt 

In [38]:
embedding_dim = 48
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [39]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [40]:
word2vec.fit(dataset, epochs=50, callbacks=[tensorboard_callback])

Epoch 1/50
49020/49020 ━━━━━━━━━━━━━━━━━━━━ 361s 7ms/step - accuracy: 0.5895 - loss: 1.0613
Epoch 2/50
49020/49020 ━━━━━━━━━━━━━━━━━━━━ 350s 7ms/step - accuracy: 0.5973 - loss: 1.0306
Epoch 3/50
49020/49020 ━━━━━━━━━━━━━━━━━━━━ 349s 7ms/step - accuracy: 0.6085 - loss: 1.0099
Epoch 4/50
49020/49020 ━━━━━━━━━━━━━━━━━━━━ 348s 7ms/step - accuracy: 0.6189 - loss: 0.9922
Epoch 5/50
49020/49020 ━━━━━━━━━━━━━━━━━━━━ 347s 7ms/step - accuracy: 0.6255 - loss: 0.9817
Epoch 6/50
49020/49020 ━━━━━━━━━━━━━━━━━━━━ 349s 7ms/step - accuracy: 0.6299 - loss: 0.9751
Epoch 7/50
49020/49020 ━━━━━━━━━━━━━━━━━━━━ 349s 7ms/step - accuracy: 0.6331 - loss: 0.9705
Epoch 8/50
49020/49020 ━━━━━━━━━━━━━━━━━━━━ 348s 7ms/step - accuracy: 0.6355 - loss: 0.9671
Epoch 9/50
49020/49020 ━━━━━━━━━━━━━━━━━━━━ 349s 7ms/step - accuracy: 0.6374 - loss: 0.9644
Epoch 10/50
49020/49020 ━━━━━━━━━━━━━━━━━━━━ 348s 7ms/step - accuracy: 0.6388 - loss: 0.9623
Epoch 11/50
49020/49020 ━━━━━━━━━━━━━━━━━━━━ 346s 7ms/step - accuracy: 0.6401 -

In [41]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [48]:
joblib_list = [weights, vocab]

joblib.dump(joblib_list, "weights_vocab.joblib")

['weights_vocab.joblib']

In [42]:
num_tokens = min(len(vocab), len(weights))

with io.open('vectors.tsv', 'w', encoding='utf-8') as out_v, \
     io.open('metadata.tsv', 'w', encoding='utf-8') as out_m:

    # Skip token 0
    for index in range(1, num_tokens):
        word = vocab[index]
        vec = weights[index]
        
        clean_word = word.strip()
        
        if not clean_word:
            clean_word = "[UNK_EMPTY]"

        # Vector
        out_v.write('\t'.join([str(x) for x in vec]) + "\n")
        
        # Metadata
        out_m.write(clean_word + "\n")

print(f"{num_tokens - 1} vectors en metadata entries.")

19999 vectors en metadata entries.


## Trainen van het model:

In [43]:
# model = EmbeddingModel.EmbeddingModel()
# model.load_dataset()
# model.create_vocab()
# model.vectorize()
# model.generate_trainingdata()
# model.word2vec()
# model.fit()
# model.save2file()

## Data:
Als trainingsdata gebruik ik 869597 Pubmed artikelen (~70GB) die ik gedownload heb van https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/oa_comm/xml/

oa_comm_xml.PMC000xxxxxx.baseline.2025-06-26.tar.gz t/m oa_comm_xml.PMC004xxxxxx.baseline.2025-06-26.tar.gz

Deze zijn nieuwer dan de artikelen uit 2021 die op /commons/data/NCBI/PubMed/ staan.

Vervolgens moeten deze artikelen doorzocht worden op inhoud. \
De .xml bestanden hebben headers voor titel, abstract, body etc. deze doorzoek ik met behulp van mijn klasse: data_parser.py

Het filteren op de drie keywords "cancer", "melanoma" en "carcinoma" duurde 9 uur en 30 minuten en reduceerde de hoeveelheid artikelen 
van 869597 naar 100914 (2.7GB).

100914 Artikelen met deze zoekcriteria opgeslagen in: /run/media/floris/FLORIS_3/Data_set/PubMed/dataset_large.txt

Voor lateren trainingsruns heb ik de data_parser.py aangepast zodat hij inplaats van het hele artikel op een enkele regel zet, een alinea per regel zet.


In [44]:
keyword_list = ["cancer", "melanoma", "carcinoma"] # Niet hoofdlettergevoelig...

trainings_data = xml_parser("/home/floris/Documenten/Dataset/PMC000xxxxxx", "/home/floris/Documenten/Dataset/trainingsData.txt", keyword_list)

# Filter de data:
trainings_data.run()



3028 XML bestanden...


100%|██████████| 3028/3028 [00:50<00:00, 59.61it/s]

Voltooid...
194 Artikelen met deze zoekcriteria opgeslagen in: /home/floris/Documenten/Dataset/trainingsData.txt


## Resultaten:

Het eerste model was met de volgende hyperparameters getrained:

vocabulary grootte: 20000 \
context lengte: 100 \
window_size=5 \
num_ns=4 \
dims = 128

Het model was erg overfit op de trainingsdata, de loss was bijna 0 en accuratesse bijna 1. \
De volgende poging zal een minder grote vocabulary krijgen en minder dimenties om beter te generaliseren i.p.v. de trainingsdata te onthouden. 

De onderstaande afbeelding weergeeft de loss en accuracy grafieken met Tensorboard:

![](img/run_1.png)


### Tweede run:

Na het proberen van vele combinaties van hyperparameters lijken deze een veel beter resultaat te geven vergeleken met de vorige:

vocabulary grootte: 1000 \
context lengte: 1000 \
window_size=5 \
num_ns=4 \
dims = 64

![](img/run_2.png)


### Derde run:


vocabulary grootte: 20000 \
context lengte: 300 \
window_size=5 \
num_ns=5 \
dims = 64


![run_3.png](img/run_3.png)


Deze laatste trainingsronde heb ik met een kleinere subset van de artikelen gemaakt dan de eerste. Het gefilterde .txt bestand bevatte maar 197 artikelen.

Het aantal regels: `wc -l trainingData.txt` = 6934 regels \
Aantal woorden `wc -w trainingData.txt` = 607965 woorden \

Voor de keywords waarop ik gefilterd heb, "cancer", "melanoma" en "carcinoma", dit is hoeveel zinnen deze woorden bevatten: \
`grep -o "cancer" trainingsData.txt | wc -l` = 4511 \
`grep -o "melanoma" trainingsData.txt | wc -l` = 588 \
`grep -o "carcinoma" trainingsData.txt | wc -l` = 1137 \
 

### Frequentie analyse trainingstekst:

`cat trainingsData.txt | tr -s ' ' '\n' | sort | uniq -c | sort -nr | head -50` =

```
  27802 the
  24475 of
  17275 and
  14315 in
  11189 to
   8467 a
   6657 with
   6243 for
   5475 were
   5346 was
   4782 that
   4563 is
   4321 The
   3910 by
   3545 as
   3086 cancer
   2950 or
   2854 be
   2826 from
   2671 are
   2635 cells
   2347 on
   2341 at
   2235 patients
   2149 not
   2129 cell
   2056 tumor
   1880 this
   1826 have
   1703 In
   1675 an
   1482 expression
   1365 which
   1322 been
   1258 these
   1203 between
   1192 we
   1189 may
   1175 has
   1112 using
   1111 than
   1105 study
   1103 also
   1094 data
    986 breast
    979 used
    918 This
    915 more
    910 all
    901 can

```

Dit bovenstaande bash statement wordt elk woord in een nieuwe regel gezet, het blijkt dus dat de "stopwoorden" die in normale zinnen voorkomen. Dit is voor de 50 meest frequent voorkomende woorden gedaan. 
Ik heb uit verschillende bronnen tegensprekende informatie gevonden ([Verminderen van stopwoorden]("https://proceedings.neurips.cc/paper_files/paper/2013/file/9aa42b31882ec039965f3c4923ce901b-Paper.pdf") en [Niet verwijderen van stopwoorden]("https://maartengr.github.io/BERTopic/getting_started/tips_and_tricks/tips_and_tricks.html")) over het nut van het verwijderen van stopwoorden. \
Uiteindelijk heb ik er voor gekozen om ze niet te verwijderen omdat deze woorden toch waarschijnlijk een belangrijke rol spelen in de relaties tussen woorden in zinnen. \
Ook het keyword "cancer" waar ik op gefilderd heb komt voor in de top 50, mijn speculatie is dat het nu zo onevenredig vaak voorkomt dat het woord geassocieerd wordt met te veel andere woorden. 



# Conclusie:

